# Análisis de Aceptación de Préstamos Personales - Naive Bayes

**Problema del libro: Data Mining for Business Analytics**

El archivo UniversalBank.csv contiene datos de 5000 clientes del Universal Bank. Entre estos 5000 clientes, solo 480 (=9.6%) aceptaron el préstamo personal que se les ofreció en la campaña anterior.

En este ejercicio nos enfocamos en dos predictores:
- **Online**: si el cliente es usuario activo de servicios bancarios en línea
- **CreditCard (CC)**: si el cliente tiene una tarjeta de crédito emitida por el banco

Y el resultado:
- **Personal Loan (Loan)**: si el cliente aceptó el préstamo personal


## 1. Importar librerías necesarias


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Instalar dmba si no está instalado
try:
    from dmba import classificationSummary, gainsChart
except ImportError:
    !pip install dmba
    from dmba import classificationSummary, gainsChart



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ------- -------------------------------- 2.4/11.8 MB 11.2 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/11.8 MB 13.6 MB/s eta 0:00:01
   ----------------------------------- ---- 10.5/11.8 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 14.8 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [graphviz]
   ---------------------------------------- 0/2 [graphviz]
   ---------------------------------------- 0/2 [graphviz]
   ---------------------------------------- 0/2 [graphviz]
   ---------------------------------------- 0/2 [graphviz]
   ---------------------------------------- 0/2 [graphviz]
   ---------------------------------------- 0/2 [graphviz]
   -------------------- ------------------- 1/2 [dmba]
   -------------------- ------------------- 1/2 [dmba]
   -----------

## 2. Cargar y examinar los datos


In [2]:
# Cargar los datos
bank_df = pd.read_csv('UniversalBank.csv')

# Examinar la estructura de los datos
print("Forma de los datos:", bank_df.shape)
print("\nPrimeras 5 filas:")
print(bank_df.head())

print("\nTipos de datos:")
print(bank_df.dtypes)

print("\nEstadísticas descriptivas de las variables relevantes:")
print(bank_df[['Online', 'CreditCard', 'Personal Loan']].describe())

print("\nDistribución de Personal Loan:")
print(bank_df['Personal Loan'].value_counts())
print("\nProporción de aceptación de préstamos:")
print(bank_df['Personal Loan'].value_counts(normalize=True))


Forma de los datos: (5000, 14)

Primeras 5 filas:
   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  

Tipos de datos:
ID                      in

## 3. Particionar los datos en entrenamiento (60%) y validación (40%)


In [ ]:
# Seleccionar las variables relevantes
predictors = ['Online', 'CreditCard']
outcome = 'Personal Loan'

# Las variables Online y CreditCard ya son binarias (0/1), no necesitan get_dummies
X = bank_df[predictors]
y = bank_df[outcome]

# Particionar los datos
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

print("Tamaño del conjunto de entrenamiento:", X_train.shape[0])
print("Tamaño del conjunto de validación:", X_valid.shape[0])

print("\nColumnas en X_train:")
print(X_train.columns.tolist())

print("\nDistribución de Personal Loan en entrenamiento:")
print(y_train.value_counts())
print("\nProporción en entrenamiento:")
print(y_train.value_counts(normalize=True))

print("\nDistribución de Personal Loan en validación:")
print(y_valid.value_counts())
print("\nProporción en validación:")
print(y_valid.value_counts(normalize=True))


Tamaño del conjunto de entrenamiento: 3000
Tamaño del conjunto de validación: 2000

Columnas en X_train:
['Online', 'CreditCard']

Distribución de Personal Loan en entrenamiento:
Personal Loan
0    2713
1     287
Name: count, dtype: int64

Proporción en entrenamiento:
Personal Loan
0    0.904333
1    0.095667
Name: proportion, dtype: float64

Distribución de Personal Loan en validación:
Personal Loan
0    1807
1     193
Name: count, dtype: int64

Proporción en validación:
Personal Loan
0    0.9035
1    0.0965
Name: proportion, dtype: float64


## 4. Crear tabla pivot para los datos de entrenamiento

### a) Tabla pivot con Online como variable de columna, CC como variable de fila y Loan como variable de fila secundaria


In [ ]:
# Crear un DataFrame de entrenamiento con las variables relevantes
train_df = pd.DataFrame({
    'Online': X_train['Online'],
    'CC': X_train['CreditCard'],
    'Loan': y_train
})

print("Primeras filas del DataFrame de entrenamiento:")
print(train_df.head())

# Crear tabla pivot usando melt() y pivot()
# Primero, crear una tabla con conteos
pivot_data = train_df.groupby(['CC', 'Online', 'Loan']).size().reset_index(name='Count')

print("\nDatos agrupados:")
print(pivot_data)

# Usar pivot para crear la tabla final
pivot_table = pivot_data.pivot_table(
    index=['CC', 'Loan'], 
    columns='Online', 
    values='Count', 
    fill_value=0
)

print("\nTabla pivot con Online como columnas, CC y Loan como filas:")
print(pivot_table)

# Mostrar también la tabla con nombres más claros
pivot_table_named = pivot_table.copy()
pivot_table_named.index.names = ['CreditCard', 'PersonalLoan']
pivot_table_named.columns = ['Online=0', 'Online=1']
print("\nTabla pivot con nombres descriptivos:")
print(pivot_table_named)


Primeras filas del DataFrame de entrenamiento:
      Online  CC  Loan
4522       0   0     0
2851       1   0     0
2313       1   1     0
982        0   1     0
1164       1   0     1

Datos agrupados:
   CC  Online  Loan  Count
0   0       0     0    792
1   0       0     1     73
2   0       1     0   1117
3   0       1     1    126
4   1       0     0    327
5   1       0     1     39
6   1       1     0    477
7   1       1     1     49

Tabla pivot con Online como columnas, CC y Loan como filas:
Online       0       1
CC Loan               
0  0     792.0  1117.0
   1      73.0   126.0
1  0     327.0   477.0
   1      39.0    49.0

Tabla pivot con nombres descriptivos:
                         Online=0  Online=1
CreditCard PersonalLoan                    
0          0                792.0    1117.0
           1                 73.0     126.0
1          0                327.0     477.0
           1                 39.0      49.0


### b) Calcular P(Loan = 1 | CC = 1, Online = 1)


In [8]:
# Calcular P(Loan = 1 | CC = 1, Online = 1) desde la tabla pivot
# Esto es el conteo de (CC=1, Online=1, Loan=1) dividido por el total de (CC=1, Online=1)

# Contar casos donde CC=1, Online=1, Loan=1
loan_accept_cc_online = pivot_table.loc[(1, 1), 1]  # CC=1, Loan=1, Online=1

# Contar total de casos donde CC=1, Online=1
total_cc_online = pivot_table.loc[(1, 0), 1] + pivot_table.loc[(1, 1), 1]  # CC=1, Online=1 (tanto Loan=0 como Loan=1)

prob_loan_given_cc_online = loan_accept_cc_online / total_cc_online

print(f"Casos con CC=1, Online=1, Loan=1: {loan_accept_cc_online}")
print(f"Total casos con CC=1, Online=1: {total_cc_online}")
print(f"P(Loan = 1 | CC = 1, Online = 1) = {prob_loan_given_cc_online:.4f}")

# Verificar con cálculo directo
mask = (train_df['CC'] == 1) & (train_df['Online'] == 1)
subset = train_df[mask]
prob_direct = subset['Loan'].mean()
print(f"\nVerificación directa: P(Loan = 1 | CC = 1, Online = 1) = {prob_direct:.4f}")


Casos con CC=1, Online=1, Loan=1: 49.0
Total casos con CC=1, Online=1: 526.0
P(Loan = 1 | CC = 1, Online = 1) = 0.0932

Verificación directa: P(Loan = 1 | CC = 1, Online = 1) = 0.0932


### c) Crear dos tablas pivot separadas


In [9]:
# Tabla pivot 1: Loan (filas) como función de Online (columnas)
pivot_loan_online = train_df.groupby(['Loan', 'Online']).size().unstack(fill_value=0)
print("Tabla pivot: Loan vs Online")
print(pivot_loan_online)

# Tabla pivot 2: Loan (filas) como función de CC (columnas)
pivot_loan_cc = train_df.groupby(['Loan', 'CC']).size().unstack(fill_value=0)
print("\nTabla pivot: Loan vs CreditCard")
print(pivot_loan_cc)


Tabla pivot: Loan vs Online
Online     0     1
Loan              
0       1119  1594
1        112   175

Tabla pivot: Loan vs CreditCard
CC       0    1
Loan           
0     1909  804
1      199   88


### d) Calcular las probabilidades condicionales requeridas


In [10]:
# Calcular todas las probabilidades requeridas

# P(CC = 1 | Loan = 1) - proporción de titulares de tarjeta de crédito entre los que aceptaron préstamos
p_cc1_given_loan1 = pivot_loan_cc.loc[1, 1] / pivot_loan_cc.loc[1, :].sum()

# P(Online = 1 | Loan = 1)
p_online1_given_loan1 = pivot_loan_online.loc[1, 1] / pivot_loan_online.loc[1, :].sum()

# P(Loan = 1) - proporción de aceptadores de préstamos
p_loan1 = y_train.sum() / len(y_train)

# P(CC = 1 | Loan = 0)
p_cc1_given_loan0 = pivot_loan_cc.loc[0, 1] / pivot_loan_cc.loc[0, :].sum()

# P(Online = 1 | Loan = 0)
p_online1_given_loan0 = pivot_loan_online.loc[0, 1] / pivot_loan_online.loc[0, :].sum()

# P(Loan = 0)
p_loan0 = 1 - p_loan1

print("Probabilidades calculadas:")
print(f"P(CC = 1 | Loan = 1) = {p_cc1_given_loan1:.4f}")
print(f"P(Online = 1 | Loan = 1) = {p_online1_given_loan1:.4f}")
print(f"P(Loan = 1) = {p_loan1:.4f}")
print(f"P(CC = 1 | Loan = 0) = {p_cc1_given_loan0:.4f}")
print(f"P(Online = 1 | Loan = 0) = {p_online1_given_loan0:.4f}")
print(f"P(Loan = 0) = {p_loan0:.4f}")


Probabilidades calculadas:
P(CC = 1 | Loan = 1) = 0.3066
P(Online = 1 | Loan = 1) = 0.6098
P(Loan = 1) = 0.0957
P(CC = 1 | Loan = 0) = 0.2964
P(Online = 1 | Loan = 0) = 0.5875
P(Loan = 0) = 0.9043


### e) Calcular P(Loan = 1 | CC = 1, Online = 1) usando Naive Bayes


In [11]:
# Aplicar la fórmula de Naive Bayes
# P(Loan = 1 | CC = 1, Online = 1) = P(CC = 1 | Loan = 1) * P(Online = 1 | Loan = 1) * P(Loan = 1) / P(CC = 1, Online = 1)

# Calcular P(CC = 1, Online = 1) usando la ley de probabilidad total
p_cc1_online1 = (p_cc1_given_loan1 * p_online1_given_loan1 * p_loan1 + 
                 p_cc1_given_loan0 * p_online1_given_loan0 * p_loan0)

# Calcular P(Loan = 1 | CC = 1, Online = 1) usando Naive Bayes
p_loan1_given_cc1_online1_naive = (p_cc1_given_loan1 * p_online1_given_loan1 * p_loan1) / p_cc1_online1

print("Cálculo usando Naive Bayes:")
print(f"P(CC = 1, Online = 1) = {p_cc1_online1:.4f}")
print(f"P(Loan = 1 | CC = 1, Online = 1) = {p_loan1_given_cc1_online1_naive:.4f}")

print(f"\nComparación:")
print(f"Desde tabla pivot: {prob_loan_given_cc_online:.4f}")
print(f"Desde Naive Bayes: {p_loan1_given_cc1_online1_naive:.4f}")
print(f"Diferencia: {abs(prob_loan_given_cc_online - p_loan1_given_cc1_online1_naive):.4f}")


Cálculo usando Naive Bayes:
P(CC = 1, Online = 1) = 0.1753
P(Loan = 1 | CC = 1, Online = 1) = 0.1020

Comparación:
Desde tabla pivot: 0.0932
Desde Naive Bayes: 0.1020
Diferencia: 0.0088


### f) Implementar Naive Bayes con scikit-learn


In [ ]:
# Entrenar el modelo Naive Bayes
nb_model = MultinomialNB(alpha=0.01)
nb_model.fit(X_train, y_train)

# Predecir probabilidades en el conjunto de entrenamiento
pred_proba_train = nb_model.predict_proba(X_train)

# Crear un DataFrame con las probabilidades predichas
prob_df = pd.DataFrame(pred_proba_train, columns=['P(Loan=0)', 'P(Loan=1)'], index=X_train.index)
prob_df['Online'] = X_train['Online']
prob_df['CreditCard'] = X_train['CreditCard']
prob_df['Actual'] = y_train

# Encontrar la probabilidad para el caso específico: CC=1, Online=1
mask = (prob_df['CreditCard'] == 1) & (prob_df['Online'] == 1)
specific_case = prob_df[mask]

print("Casos con CC=1, Online=1:")
print(specific_case[['P(Loan=0)', 'P(Loan=1)', 'Actual']].head(10))

if len(specific_case) > 0:
    sklearn_prob = specific_case['P(Loan=1)'].iloc[0]  # Tomar el primer caso como ejemplo
    print(f"\nP(Loan = 1 | CC = 1, Online = 1) desde scikit-learn: {sklearn_prob:.4f}")
    print(f"\nComparación final:")
    print(f"Tabla pivot: {prob_loan_given_cc_online:.4f}")
    print(f"Naive Bayes manual: {p_loan1_given_cc1_online1_naive:.4f}")
    print(f"Scikit-learn: {sklearn_prob:.4f}")
else:
    print("No se encontraron casos con CC=1, Online=1 en el conjunto de entrenamiento")


Casos con CC=1, Online=1:
      P(Loan=0)  P(Loan=1)  Actual
2313   0.904419   0.095581       0
1918   0.904419   0.095581       1
4506   0.904419   0.095581       0
586    0.904419   0.095581       0
3591   0.904419   0.095581       0
2166   0.904419   0.095581       0
1335   0.904419   0.095581       0
4955   0.904419   0.095581       0
734    0.904419   0.095581       0
4576   0.904419   0.095581       0

P(Loan = 1 | CC = 1, Online = 1) desde scikit-learn: 0.0956

Comparación final:
Tabla pivot: 0.0932
Naive Bayes manual: 0.1020
Scikit-learn: 0.0956


### g) Evaluación del modelo


In [ ]:
# Predecir en el conjunto de validación
y_valid_pred = nb_model.predict(X_valid)
pred_proba_valid = nb_model.predict_proba(X_valid)

# Mostrar resumen de clasificación
print("Resumen de clasificación en conjunto de validación:")
classificationSummary(y_valid, y_valid_pred)

# Mostrar algunas predicciones de probabilidad
print("\nPrimeras 10 predicciones de probabilidad:")
prob_valid_df = pd.DataFrame(pred_proba_valid, columns=['P(Loan=0)', 'P(Loan=1)'])
prob_valid_df['Predicted'] = y_valid_pred
prob_valid_df['Actual'] = y_valid.values
print(prob_valid_df.head(10))

# Buscar casos específicos en validación
valid_df = pd.DataFrame({
    'Online': X_valid['Online'],
    'CreditCard': X_valid['CreditCard'],
    'Actual': y_valid.values,
    'Predicted': y_valid_pred,
    'P(Loan=0)': pred_proba_valid[:, 0],
    'P(Loan=1)': pred_proba_valid[:, 1]
})

# Casos con CC=1, Online=1 en validación
valid_specific = valid_df[(valid_df['CreditCard'] == 1) & (valid_df['Online'] == 1)]
print(f"\nCasos con CC=1, Online=1 en validación: {len(valid_specific)}")
if len(valid_specific) > 0:
    print(valid_specific[['Actual', 'Predicted', 'P(Loan=0)', 'P(Loan=1)']].head())


Resumen de clasificación en conjunto de validación:
Confusion Matrix (Accuracy 0.9035)

       Prediction
Actual    0    1
     0 1807    0
     1  193    0

Primeras 10 predicciones de probabilidad:
   P(Loan=0)  P(Loan=1)  Predicted  Actual
0   0.904506   0.095494          0       0
1   0.904333   0.095667          0       0
2   0.904333   0.095667          0       0
3   0.904246   0.095754          0       0
4   0.904246   0.095754          0       0
5   0.904246   0.095754          0       0
6   0.904246   0.095754          0       0
7   0.904506   0.095494          0       0
8   0.904419   0.095581          0       0
9   0.904506   0.095494          0       0

Casos con CC=1, Online=1 en validación: 356
      Actual  Predicted  P(Loan=0)  P(Loan=1)
932        0          0   0.904419   0.095581
1132       0          0   0.904419   0.095581
3289       0          0   0.904419   0.095581
348        1          0   0.904419   0.095581
2971       0          0   0.904419   0.095581


## Conclusiones

### Respuestas a las preguntas del ejercicio:

**b) Probabilidad desde tabla pivot:** La probabilidad P(Loan = 1 | CC = 1, Online = 1) calculada directamente desde la tabla pivot es la estimación más precisa porque utiliza todos los datos disponibles sin asumir independencia entre las variables.

**f) Comparación de estimaciones:** La estimación desde la tabla pivot es más precisa que la de Naive Bayes porque Naive Bayes asume independencia condicional entre las variables predictoras, lo cual puede no ser cierto en la realidad.

**g) Entradas necesarias:** Para calcular P(Loan = 1 | CC = 1, Online = 1) usando Naive Bayes, necesitamos:
- P(CC = 1 | Loan = 1)
- P(Online = 1 | Loan = 1) 
- P(Loan = 1)
- P(CC = 1 | Loan = 0)
- P(Online = 1 | Loan = 0)
- P(Loan = 0)

El modelo de scikit-learn proporciona una implementación robusta de Naive Bayes que maneja automáticamente estos cálculos.
